# Some exploratory data anaylisis, to check that the normalisation is working

In [ ]:
from configs import avail_models, get_avail_splits
from models import norm_vals
from video_dataset import get_wlasl_info, get_data_set
import utils
import configs
import torch

In [2]:
configs.set_seed()

## Setup

### Model

Available models:

In [3]:
av_models = avail_models()
for m in av_models:
    print(m)

S3D
R3D_18
R(2+1)D_18
Swin3D_T
Swin3D_S
Swin3D_B
MViTv2_S
MViTv1_B


Chosen model: 

In [4]:
model_name = av_models[0]
print(model_name)

S3D


Norm values: 

In [5]:
model_info = norm_vals(model_name)
print(model_info)

{'mean': (0.43216, 0.394666, 0.37645), 'std': (0.22803, 0.22145, 0.216989)}


### Dataset

Available splits

In [6]:
av_splits = get_avail_splits()
for s in av_splits:
    print(s)

asl100
asl2000
asl300
asl1000


Chosen split, and set

In [7]:
split_name = av_splits[0]
print(split_name)

asl100


Dataset info

In [8]:
train_info = get_wlasl_info(split_name, set_name="train")
print(train_info)


{'root': PosixPath('../data/WLASL/WLASL2000'), 'labels': PosixPath('preprocessed/labels/asl100'), 'label_suff': 'fixed_frange_bboxes_len.json', 'set_name': 'train'}


Class list

In [9]:
class_list = configs.get_class_list()
print(len(class_list))

2000


#### Data Sets

one in regular conditions, and the other 'raw'


In [10]:
model_info = norm_vals(model_name)
print(model_info)
num_frames = 16
frame_size = 112
batch_size = 1

{'mean': (0.43216, 0.394666, 0.37645), 'std': (0.22803, 0.22145, 0.216989)}


In [ ]:
train_set, num_classes, _, _ = get_data_set(
    mean=model_info['mean'],
    std=model_info['std'],
    num_frames=num_frames,
    frame_size=frame_size,
    set_info=train_info,
)

raw_set, _, _, _ = get_data_set(
    mean=model_info['mean'],
    std=model_info['std'],
    num_frames=num_frames,
    frame_size=frame_size,
    set_info=train_info,
    all_frames=True,
    do_crop=False,
    do_norm=False)

In [12]:
print(len(train_loader))
print(num_classes)

1442
100


## Examine frames

In [13]:
iter_tloader = iter(train_loader)
iter_rloader = iter(raw_loader)
sample = next(iter_tloader)
sample_raw = next(iter_rloader)

In [14]:
rf = sample_raw['frames']
print(rf.shape)

torch.Size([1, 3, 16, 256, 256])


In [15]:
def prep_next(sample: dict[str, int | torch.Tensor]) -> tuple[torch.Tensor, int]:
    frames = sample['frames']
    assert isinstance(frames, torch.Tensor), f'frames is type: {type(frames)}'
    label_num = sample['label_num']
    assert isinstance(label_num, torch.Tensor), f'label num is type: {type(label_num)}'
    
    frames = frames.squeeze()
    frames = frames.permute(1, 0, 2, 3)
    return frames, int(label_num.squeeze())
    

In [ ]:
def prep_next2(sample: dict[str, int | torch.Tensor]) -> tuple[torch.Tensor, int]:
    frames = sample['frames']
    assert isinstance(frames, torch.Tensor), f'frames is type: {type(frames)}'
    label_num = sample['label_num']
    assert isinstance(label_num, torch.Tensor), f'label num is type: {type(label_num)}'
    
    if len(frames.shape) == 5:
        frames = frames.squeeze(dim=0)
    if frames.shape[1] != 3: 
        frames = frames.permute(1, 0, 2, 3) #swap T and C
    return frames, int(label_num.squeeze())

In [23]:
frames, label_num = prep_next2(sample_raw)
print(f"frames: {frames.shape}")
print(f'Class: {class_list[label_num]}')
utils.plt_display(frames, num_frames, adapt=True)

5
frames: torch.Size([1, 3, 16, 256, 256])
Class: can


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 4 is not equal to len(dims) = 3

In [ ]:
rframes, rlabel_num = prep_next(sample)
print(f"frames: {frames.shape}")
print(f'Class: {class_list[label_num]}')
utils.plt_display(rframes, num_frames, adapt=True)

NameError: name 'prep_next' is not defined